In [ ]:
# ! pip install ipykernel
# ! pip install gradio
# ! pip install -U langchain-openai
# ! pip install ipywidgets --upgrade
# ! pip install --upgrade langchain
# ! pip install langchain_community

In [ ]:
import json
import os
import uuid
from datetime import datetime
import gradio as gr
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage

In [ ]:
OPENAI_API_KEY = os.environ.get ("openai_api_key")

In [ ]:
# # Retrieve the list of available models
# client = OpenAI(api_key = OPENAI_API_KEY)
# models = client.models.list()
# for model in models.data:
#     print(model.id)

In [ ]:
# Langchain opeai 모델 초기화
TEMPERATURE = 0
MODEL_NAME = 'gpt-4o'
llm = ChatOpenAI(temperature=TEMPERATURE, model=MODEL_NAME)

# 전체 대화 기록을 유지할 전역 변수
global_chat_history = {
    "meta": {
        "temperature": TEMPERATURE,
        "model": MODEL_NAME
    },
    "prompts": []
}

# 현재 시간을 yyyymmdd_hhmmss 형식으로 반환하는 함수
def get_current_time():
    return datetime.now().strftime("%Y%m%d_%H%M%S")

# 로컬에 대화 기록 저장 함수 (버튼 클릭 시 호출)
def save_chat_history_to_local(session_id):
    output_directory = "_output_OpenAI"
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    timestamp = get_current_time()
    file_name = f"{timestamp}_{session_id}.json"
    file_path = os.path.join(output_directory, file_name)

    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(global_chat_history, file, ensure_ascii=False, indent=4)
    print(f"Chat history saved to {file_path}")
    return f"Chat history saved to {file_path}"

# Gradio에서 유저의 질문과 채팅기록을 받아 GPT-4의 대답을 리턴하는 함수
def response(message, history, system_message):
    history_langchain_format = [SystemMessage(content=system_message)]
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))
    gpt_response = llm.invoke(history_langchain_format)
    user_message_time = get_current_time()
    bot_response_time = get_current_time()
    
    # 글로벌 대화 기록에 추가
    if not global_chat_history["prompts"] or global_chat_history["prompts"][-1]["prompt"] != system_message:
        global_chat_history["prompts"].append({
            "prompt": system_message,
            "history": []
        })
    
    global_chat_history["prompts"][-1]["history"].append({
        "user_message": message,
        "user_message_time": user_message_time,
        "bot_response": gpt_response.content,
        "bot_response_time": bot_response_time
    })
    
    return gpt_response.content

# Gradio 세션 관리 함수
def generate_session_id():
    return str(uuid.uuid4())

# Gradio UI 설정 및 실행
with gr.Blocks() as demo:
    session_id_input = gr.Textbox(value=generate_session_id(), label="Session ID", placeholder="Enter your session ID")
    chatbot = gr.Chatbot(height=500)
    msg = gr.Textbox(placeholder="User Message를 입력해 주세요", container=False, scale=7)
    additional_input = gr.Textbox("", label="System Prompt", placeholder="")
    save_button = gr.Button("대화 기록 저장")

    state = gr.State(value="")  # 초기 시스템 프롬프트 상태

    def submit_message(message, history, session_id, system_message):
        response_text = response(message, history, system_message)
        history.append((message, response_text))
        return history, ""
    
    def delete_last_chat(chat_history):
        if len(chat_history) > 0:
            chat_history.pop()
        return chat_history

    def update_system_message(new_prompt):
        state.value = new_prompt
        return new_prompt

    def save_chat(session_id):
        return save_chat_history_to_local(session_id)

    msg.submit(submit_message, [msg, chatbot, session_id_input, state], [chatbot, msg])
    additional_input.change(update_system_message, inputs=additional_input, outputs=state)
    save_button.click(save_chat, inputs=[session_id_input], outputs=None)

    gr.Button("입력하기 ↩").click(fn=submit_message, inputs=[msg, chatbot, session_id_input, state], outputs=[chatbot, msg])
    gr.Button("이전챗 삭제 ❌").click(fn=delete_last_chat, inputs=chatbot, outputs=chatbot)
    gr.Button("전챗 삭제 💫").click(fn=lambda: [], inputs=None, outputs=chatbot)

demo.launch(share=True)
